In [3]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
boston=load_boston()
dfX=pd.DataFrame(boston.data, columns=boston.feature_names)
dfy=pd.DataFrame(boston.target, columns=["MEDV"])
df=pd.concat([dfX, dfy], axis=1)

c:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :fu

In [4]:
#학습용과 검증용을 7:3으로 구분
N=len(df)
ratio=0.7
np.random.seed(0)
idx_train=np.random.choice(np.arange(N), np.int64(ratio*N),replace=False)
idx_test=list(set(np.arange(N)).difference(idx_train))
            # 집합               차집합     학습용 70%
df_train=df.iloc[idx_train]
df_test=df.iloc[idx_test]

In [5]:
boston

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [6]:
import statsmodels.api as sm
model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     70.06
Date:                Thu, 09 Mar 2023   Prob (F-statistic):           8.57e-88
Time:                        11:05:12   Log-Likelihood:                -1043.0
No. Observations:                 354   AIC:                             2114.
Df Residuals:                     340   BIC:                             2168.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.0719      5.932      5.913      0.0

In [7]:
#검증용 데이터셋으로 모형 평가
pred=result.predict(df_test)
# Residual Sum of Square(잔차의 분산, 오차의 크기)
ssr=((df_test.MEDV-pred) ** 2).sum()
# Total Sum of Square(종속변수 y의 분산)
sst=((df_test.MEDV-df_test.MEDV.mean())**2).sum()
# 결정계수: 모형의 설명력(0~1 사이의 값)
rsquared=1-ssr/sst
rsquared

0.7519796502601109

In [8]:
from sklearn.model_selection import train_test_split
#학습용,검증용으로 구분
df_train, df_test=train_test_split(df, test_size=0.3, random_state=0)
df_train.shape, df_test.shape

((354, 14), (152, 14))

In [9]:
#학습용X,y 검증용 X,y로 구분
dfX_train, dfX_test, dfy_train, dfy_test=train_test_split(dfX, dfy, test_size=0.3, random_state=0)
dfX_train.shape, dfy_train.shape, dfX_test.shape, dfy_test.shape

((354, 13), (354, 1), (152, 13), (152, 1))

In [10]:
from sklearn.model_selection import KFold

scores=np.zeros(5)
cv=KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train=df.iloc[idx_train]
    df_test=df.iloc[idx_test]

    model=sm.OLS.from_formula("MEDV~"+"+".join(boston.feature_names), data=df_train)
    result=model.fit()

    pred=result.predict(df_test)
    ssr=((df_test.MEDV-pred)**2).sum()
    sst=((df_test.MEDV-df_test.MEDV.mean())**2).sum()
    rsquared=1-ssr/sst

    scores[i]=rsquared
    print(f"학습용 R2={result.rsquared:.3f}, 검증용 R2={rsquared:.3f}")

학습용 R2=0.773, 검증용 R2=0.589
학습용 R2=0.729, 검증용 R2=0.778
학습용 R2=0.749, 검증용 R2=0.668
학습용 R2=0.757, 검증용 R2=0.668
학습용 R2=0.705, 검증용 R2=0.840


In [11]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
scores1=np.zeros(5)
scores2=np.zeros(5)
scores3=np.zeros(5)
cv=KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
 df_train=df.iloc[idx_train]
 df_test=df.iloc[idx_test]

 model=sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
 result=model.fit()

 pred=result.predict(df_test)
 #결정계수를 구하는 함수
 rsquared=r2_score(df_test.MEDV, pred)
 scores1[i]=rsquared
 #평균제곱오차(Mean Squared Error): 오차의 제곱의 합계의 평균값
 mse=mean_squared_error(df_test.MEDV, pred)
 scores2[i]=mse
 #평균절대오차(Mean Absolute Error): 오차의 절대값 합계의 평균값
 mae=mean_absolute_error(df_test.MEDV, pred)
 scores3[i]=mae

print(scores1) # rsquare
print(scores2) # mse
print(scores3) # mae
print(np.mean(scores1))
print(np.mean(scores2))
print(np.mean(scores3))

[0.58922238 0.77799144 0.66791979 0.6680163  0.83953317]
[33.44898    18.65881615 21.23463289 29.22251557 16.57369039]
[3.84290922 3.38979394 3.07473854 3.6463452  3.03058651]
0.7085366175182802
23.82772699990077
3.396874681157459


In [12]:
from sklearn.base import BaseEstimator, RegressorMixin
import statsmodels.formula.api as smf
import statsmodels.api as sm
class StatsmodelsOLS(BaseEstimator, RegressorMixin):
 def __init__(self, formula):
   self.formula=formula
   self.model=None
   self.data=None
   self.result=None

 def fit(self, dfX, dfy):
   self.data=pd.concat([dfX, dfy], axis=1)
   self.model=smf.ols(self.formula, data=self.data)
   self.result=self.model.fit()

 def predict(self, new_data):
   return self.result.predict(new_data)

In [13]:
from sklearn.model_selection import cross_val_score

model=StatsmodelsOLS("MEDV ~ " + "+".join(boston.feature_names))
cv=KFold(5, shuffle=True, random_state=0)
cross_val_score(model, dfX, dfy, scoring="r2", cv=cv)

#평균제곱오차로 평가하는 경우
result=cross_val_score(model, dfX, dfy, scoring='neg_mean_squared_error', cv=cv)

#음수로 나온 결과값을 양수로 변환
rmse_score=np.sqrt(-result)
rmse_score

array([5.78350932, 4.31958518, 4.60810513, 5.40578538, 4.07107976])